In [2]:
import detectron2
print(detectron2.__version__)

0.6


In [3]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

c:\Users\krris\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Disabling PyTorch because PyTorch >= 2.2 is required but found 2.1.2+cu121
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [8]:

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file(
        "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
    )
)

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
)

#cfg.MODEL.DEVICE = "cuda"
predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])


In [5]:
import time

In [9]:
video_path = r"D:\Detectron\carr.mp4" # <-- change this
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Error opening video file"


In [10]:
with torch.no_grad():  # crucial for speed
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start = time.time()

        outputs = predictor(frame)

        v = Visualizer(
            frame[:, :, ::-1],  # BGR → RGB for visualization
            metadata=metadata,
            scale=1.0
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        result = out.get_image()[:, :, ::-1].copy()  # MAKE CONTIGUOUS


        fps = 1 / (time.time() - start)
        cv2.putText(
            result,
            f"FPS: {fps:.2f}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2
        )

        cv2.imshow("Detectron2 Video Stream", result)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

KeyboardInterrupt: 